In [ ]:
from google.colab import drive
import cv2
import numpy as np
import pandas as pd
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
train_path = '/gdrive/My Drive/Colab Notebooks/Datafiles/Hackathon Scene Recognition/train1/train'

In [ ]:
train_labels = pd.read_csv("/gdrive/My Drive/Colab Notebooks/Datafiles/Hackathon Scene Recognition/train.csv")

In [ ]:
list_labels = list(train_labels['image_name'])

In [ ]:
y_train = list(train_labels['label'])

In [ ]:
str(train_labels['label'][train_labels['image_name'] == '23659.jpg']).split()[1]

'3'

In [ ]:
def create_dataset(train_path):
    new_train_path = '/gdrive/My Drive/Colab Notebooks/Datafiles/Hackathon Scene Recognition/train'
    test_path = '/gdrive/My Drive/Colab Notebooks/Datafiles/Hackathon Scene Recognition/test'
    test_images = []
    for i in range(24335):
        image_name = str(i) + '.jpg'
        image_path = train_path + '/' + image_name
        if image_name not in list_labels:
          test_images.append(image_name)
        if i%500==0:
          print(i)
          
    return test_images
test_images = create_dataset(train_path)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000


In [ ]:
IMG_HEIGHT, IMG_WIDTH = 224, 224
batch_size = 128
epochs = 50
def prep_fn(img):
    img = img.astype(np.float32) / 255.0
    img = (img - 0.5) * 2
    return img

train_image_generator = ImageDataGenerator(rotation_range=15,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                validation_split=0.2,
                                preprocessing_function=prep_fn
                                ) 
train_path = '/gdrive/My Drive/Colab Notebooks/Datafiles/Hackathon Scene Recognition/train'
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_path,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical',
                                                           subset='training')
val_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_path,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical',
                                                           subset='validation')

Found 13630 images belonging to 6 classes.
Found 3404 images belonging to 6 classes.


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout


# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = BatchNormalization()(x)
x = Dense(1024, activation='elu')(x)
x = Dropout(0.8)(x)
x = BatchNormalization()(x)
x = Dense(512, activation='elu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(128, activation='elu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)

predictions = Dense(6, activation='softmax')(x)
x = Dropout(0.3)(x)
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
model.summary()
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
epochs = 50
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.0000001)
callbacks = [earlystop,learning_rate_reduction]
# train the model on the new data for a few epochs
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch = train_data_gen.samples // batch_size,
    validation_data = val_data_gen, 
    validation_steps = val_data_gen.samples // batch_size,
    epochs = epochs, callbacks=callbacks)

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_376 (Conv2D)            (None, None, None,   864         ['input_15[0][0]']               
                                32)                                                               
                                                                                                  
 batch_normalization_385 (Batch  (None, None, None,   96         ['conv2d_376[0][0]']             
 Normalization)                 32)                                                         

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/50
106/106 [==============================] - 214s 2s/step - loss: 0.5316 - accuracy: 0.8147 - val_loss: 0.4136 - val_accuracy: 0.8642 - lr: 0.0010
Epoch 2/50
106/106 [==============================] - 206s 2s/step - loss: 0.3739 - accuracy: 0.8648 - val_loss: 0.2781 - val_accuracy: 0.8969 - lr: 0.0010
Epoch 3/50
106/106 [==============================] - 208s 2s/step - loss: 0.3404 - accuracy: 0.8788 - val_loss: 0.2998 - val_accuracy: 0.8969 - lr: 0.0010
Epoch 4/50
106/106 [==============================] - 208s 2s/step - loss: 0.3217 - accuracy: 0.8862 - val_loss: 0.2633 - val_accuracy: 0.9099 - lr: 0.0010
Epoch 5/50
106/106 [==============================] - 207s 2s/step - loss: 0.3190 - accuracy: 0.8856 - val_loss: 0.2624 - val_accuracy: 0.9041 - lr: 0.0010
Epoch 6/50
106/106 [==============================] - ETA: 0s - loss: 0.2944 - accuracy: 0.8942
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
106/106 [==============================] - 2

KeyboardInterrupt: ignored

In [ ]:
base_model.trainable = True
model.summary()
epochs = 10
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 15)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.0000001)
callbacks = [earlystop,learning_rate_reduction]
# train the model on the new data for a few epochs
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch = train_data_gen.samples // batch_size,
    validation_data = val_data_gen, 
    validation_steps = val_data_gen.samples // batch_size,
    epochs = epochs, callbacks=callbacks)

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_376 (Conv2D)            (None, None, None,   864         ['input_15[0][0]']               
                                32)                                                               
                                                                                                  
 batch_normalization_385 (Batch  (None, None, None,   96         ['conv2d_376[0][0]']             
 Normalization)                 32)                                                         

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  from ipykernel import kernelapp as app


Epoch 1/10
 10/106 [=>............................] - ETA: 2:43 - loss: 1.5309 - accuracy: 0.6054

KeyboardInterrupt: ignored

In [ ]:
import PIL
from PIL import UnidentifiedImageError
import glob

imgs_ = glob.glob("/gdrive/My Drive/Colab Notebooks/Datafiles/Hackathon Scene Recognition/train/*/*.jpg")

for img in imgs_:
    try:
        img = PIL.Image.open(img)
    except PIL.UnidentifiedImageError:
        print(img)

/gdrive/My Drive/Colab Notebooks/Datafiles/Hackathon Scene Recognition/train/2/7645.jpg


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import tensorflow as tf
from tensorflow.keras import layers

# create the base pre-trained model
inputs = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")

# Freeze the pretrained weights
model.trainable = False

# Rebuild top
x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
x = layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = layers.Dense(6, activation="softmax", name="pred")(x)
model.summary()
# Compile
model = tf.keras.Model(inputs, outputs, name="EfficientNet")
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 15)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 3,verbose = 1,factor = 0.5,min_lr = 0.0000001)
callbacks = [earlystop,learning_rate_reduction]
# train the model on the new data for a few epochs
history = model.fit(
    train_data_gen,
    steps_per_epoch = train_data_gen.samples // batch_size,
    validation_data = val_data_gen, 
    validation_steps = val_data_gen.samples // batch_size,
    epochs = epochs, callbacks=callbacks)

In [ ]:
base_model.trainable = True
model.summary()
epochs = 10
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 15)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 3,verbose = 1,factor = 0.5,min_lr = 0.0000001)
callbacks = [earlystop,learning_rate_reduction]
# train the model on the new data for a few epochs
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch = train_data_gen.samples // batch_size,
    validation_data = val_data_gen, 
    validation_steps = val_data_gen.samples // batch_size,
    epochs = epochs, callbacks=callbacks)

In [ ]:
IMG_HEIGHT, IMG_WIDTH = 224, 224
batch_size = 64
epochs = 50
def prep_fn(img):
    img = img.astype(np.float32) / 255.0
    img = (img - 0.5) * 2
    return img
import tensorflow as tf
train_image_generator = ImageDataGenerator(rotation_range=15,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                validation_split=0.2,
                                preprocessing_function=prep_fn
                                ) 
train_path = '/gdrive/My Drive/Colab Notebooks/Datafiles/Hackathon Scene Recognition/train'
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_path,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical',
                                                           subset='training')
val_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_path,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical',
                                                           subset='validation')
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout


# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = BatchNormalization()(x)
x = Dense(128, activation='elu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)

predictions = Dense(6, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
model.summary()
# compile the model (should be done *after* setting layers to non-trainable)
optimizer = tf.keras.optimizers.SGD(
    learning_rate=0.001, momentum=0.9, nesterov=False, name='SGD'
)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
epochs = 5
earlystop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15, restore_best_weights=True)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.0000001)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/model.h5",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
callbacks = [earlystop,learning_rate_reduction, model_checkpoint_callback]
# train the model on the new data for a few epochs
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch = train_data_gen.samples // batch_size,
    validation_data = val_data_gen, 
    validation_steps = val_data_gen.samples // batch_size,
    epochs = epochs, callbacks=callbacks)

Found 13630 images belonging to 6 classes.
Found 3404 images belonging to 6 classes.
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_94 (Conv2D)             (None, None, None,   864         ['input_2[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization_96 (BatchN  (None, None, None,   96         ['conv2d_94[0][0]']              
 ormali

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/5
212/212 [==============================] - 2847s 13s/step - loss: 0.4117 - accuracy: 0.8492 - val_loss: 0.2860 - val_accuracy: 0.8939 - lr: 0.0010
Epoch 2/5
212/212 [==============================] - 283s 1s/step - loss: 0.3191 - accuracy: 0.8860 - val_loss: 0.2702 - val_accuracy: 0.9036 - lr: 0.0010
Epoch 3/5
212/212 [==============================] - 309s 1s/step - loss: 0.2994 - accuracy: 0.8909 - val_loss: 0.2590 - val_accuracy: 0.9080 - lr: 0.0010
Epoch 4/5
212/212 [==============================] - 282s 1s/step - loss: 0.2821 - accuracy: 0.8972 - val_loss: 0.2546 - val_accuracy: 0.9057 - lr: 0.0010
Epoch 5/5
212/212 [==============================] - 282s 1s/step - loss: 0.2708 - accuracy: 0.9009 - val_loss: 0.2499 - val_accuracy: 0.9095 - lr: 0.0010


In [ ]:
base_model.trainable = False
fine_tune_at = 50

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = True
model.summary()
epochs = 5
batch_size = 64
optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-7)
model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 3,verbose = 1,factor = 0.5,min_lr = 0.0000001)
callbacks = [earlystop,learning_rate_reduction]
# train the model on the new data for a few epochs
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch = train_data_gen.samples // batch_size,
    validation_data = val_data_gen, 
    validation_steps = val_data_gen.samples // batch_size,
    epochs = epochs, callbacks=callbacks)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_94 (Conv2D)             (None, None, None,   864         ['input_2[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization_96 (BatchN  (None, None, None,   96         ['conv2d_94[0][0]']              
 ormalization)                  32)                                                         

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/5
 11/212 [>.............................] - ETA: 4:03 - loss: 0.8228 - accuracy: 0.7216

KeyboardInterrupt: ignored

In [ ]:
from PIL import Image
import numpy as np
test_df = pd.read_csv("/gdrive/My Drive/Colab Notebooks/Datafiles/Hackathon Scene Recognition/test_yIRt6TK.csv")
test_df['label'] = 0
preds = []
test_path = '/gdrive/My Drive/Colab Notebooks/Datafiles/Hackathon Scene Recognition/test'
for i, image in enumerate(test_images):
  try:
    im2arr = np.array(Image.open(test_path + '/' + image).resize((224, 224))).reshape(1, 224, 224, 3)/127.5-1
  except:
    print(image)
  test_df['image_name'][test_df['image_name']==image] = np.argmax(model.predict(im2arr))
  if i%500==0:
          print(i)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


0
500
1000
1500
2000
7606.jpg
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000


In [ ]:
test_df.head(20)
test_df2 = pd.read_csv("/gdrive/My Drive/Colab Notebooks/Datafiles/Hackathon Scene Recognition/test_yIRt6TK.csv")
test_df2['label'] = test_df['image_name']
test_df2.head(20)

,image_name,label
0,3.jpg,5
1,5.jpg,0
2,6.jpg,5
3,11.jpg,2
4,14.jpg,5
5,22.jpg,1
6,25.jpg,4
7,28.jpg,3
8,30.jpg,2
9,38.jpg,2


In [ ]:
test_df2.to_csv("/content/submisssion_inception.csv", index = False)

In [ ]:
print(preds, test_images[:10])

[5, 0, 5, 2, 5, 1, 4, 3, 2, 2] ['3.jpg', '5.jpg', '6.jpg', '11.jpg', '14.jpg', '22.jpg', '25.jpg', '28.jpg', '30.jpg', '38.jpg']


In [ ]:
!pip install efficientnet


     |████████████████████████████████| 50 kB 3.1 MB/s 


'2.7.0'

In [ ]:
from tensorflow.keras import Sequential

In [ ]:
IMG_HEIGHT, IMG_WIDTH = 150, 150
batch_size = 64
epochs = 50
train_image_generator = ImageDataGenerator(rotation_range=15,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                validation_split=0.2,
                                rescale=1/255.
                                ) 
train_path = '/gdrive/My Drive/Colab Notebooks/Datafiles/Hackathon Scene Recognition/train'
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_path,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical',
                                                           subset='training')
val_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_path,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical',
                                                           subset='validation')

Found 13630 images belonging to 6 classes.
Found 3404 images belonging to 6 classes.


In [ ]:
import efficientnet.keras as efn 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
import tensorflow as tf
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

base_model = efn.EfficientNetL2(weights="/gdrive/My Drive/Colab Notebooks/Datafiles/Hackathon Scene Recognition/efficientnet-l2_noisy-student_notop.h5",
                                include_top=False, drop_connect_rate=0.15, classes=6)
base_model.trainable = False

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(6, activation='softmax'))

model.summary()

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6)

#early stopping to monitor the validation loss and avoid overfitting
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)

#reducing learning rate on plateau
rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience= 5, factor= 0.5, min_lr= 1e-6, verbose=1)
callbacks = [early_stop,rlrop]
#model compiling

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
epochs = 50
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch = train_data_gen.samples // batch_size,
    validation_data = val_data_gen, 
    validation_steps = val_data_gen.samples // batch_size,
    epochs = epochs, callbacks=callbacks)

#saving the trained model weights as data file in .h5 format
model.save_weights("scene_efficientnetl2_weights.h5")

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnet-l2 (Functional  (None, None, None, 5504)  475894996
 )                                                               
                                                                 
 global_average_pooling2d_5   (None, 5504)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_7 (Dense)             (None, 6)                 33030     
                                                                 
Total params: 475,928,026
Trainable params: 33,030
Non-trainable params: 475,894,996
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/50
212/212 [==============================] - 999s 4s/step - loss: 4.3228 - accuracy: 0.1679 - val_loss: 2.4007 - val_accuracy: 0.1804 - lr: 1.0000e-06
Epoch 2/50
212/212 [==============================] - 941s 4s/step - loss: 3.0607 - accuracy: 0.1760 - val_loss: 1.9467 - val_accuracy: 0.1910 - lr: 1.0000e-06
Epoch 3/50
212/212 [==============================] - 939s 4s/step - loss: 2.2948 - accuracy: 0.1754 - val_loss: 1.7980 - val_accuracy: 0.1804 - lr: 1.0000e-06
Epoch 4/50
 65/212 [========>.....................] - ETA: 8:37 - loss: 2.0989 - accuracy: 0.1700

In [ ]:
base_model.trainable = True
model.summary()
epochs = 10
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-7)
model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 15)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 3,verbose = 1,factor = 0.5,min_lr = 0.0000001)
callbacks = [earlystop,learning_rate_reduction]
# train the model on the new data for a few epochs
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch = train_data_gen.samples // batch_size,
    validation_data = val_data_gen, 
    validation_steps = val_data_gen.samples // batch_size,
    epochs = epochs, callbacks=callbacks)